# Profile NeRF with Timeloop and Accelergy

In [1]:
import os
import re
import yaml
import json
import traceback

from profiler import Profiler
from notebook_utils import natural_sort

import sys
sys.path.append("../") # go to parent dir

from accelerating_nerfs.models import VanillaNeRF, patch_forward

## Load NeRF model
We use vanilla NeRFs which are MLPs

In [2]:
# Uncomment to view architecture diagram
# from IPython.display import IFrame
# IFrame("./figures/netdiag-modified.pdf", width=600, height=350)

In [3]:
model = VanillaNeRF()

# Need to patch the forward method for the purpose of mapping to pass in ray directions
# This ensures the bottleneck layer is captured in the timeloop outputs
patch_forward(model)
print(model)

VanillaNeRF(
  (posi_encoder): SinusoidalEncoder()
  (view_encoder): SinusoidalEncoder()
  (mlp): NerfMLP(
    (base): MLP(
      (hidden_activation): ReLU()
      (output_activation): Identity()
      (hidden_layers): ModuleList(
        (0): Linear(in_features=63, out_features=256, bias=True)
        (1): Linear(in_features=256, out_features=256, bias=True)
        (2): Linear(in_features=256, out_features=256, bias=True)
        (3): Linear(in_features=256, out_features=256, bias=True)
        (4): Linear(in_features=256, out_features=256, bias=True)
        (5): Linear(in_features=319, out_features=256, bias=True)
        (6): Linear(in_features=256, out_features=256, bias=True)
        (7): Linear(in_features=256, out_features=256, bias=True)
      )
    )
    (sigma_layer): DenseLayer(
      (hidden_activation): ReLU()
      (output_activation): Identity()
      (hidden_layers): ModuleList()
      (output_layer): Linear(in_features=256, out_features=1, bias=True)
    )
    (bottl

/home/workspace/notebooks/../accelerating_nerfs/models.py:254: UserWarning: patched forward of VanillaNeRF to also pass the condition. You should only use this for debugging or with Timeloop and Accelergy
  warnings.warn(


### Configure saving of profiling results
This isn't important so you can ignore the details.

In [4]:
# Accumulate results in this dictionary
profile_results = {}

# Setup saving the profiling results
results_dir = "profile_results"
os.makedirs(results_dir, exist_ok=True)


def save_results():
    all_other_results = {}
    
    for arch, arch_results in profile_results.items():
        # Write the super long results to it's own file
        arch_results_path = os.path.join(results_dir, f"{arch}_results.json")
        with open(arch_results_path, "w") as f:
            json.dump(arch_results["results"], f, indent=4)
            print(f"Saved {arch} results to {arch_results_path}")
        
        # Accumulate the other results as they're shorter and more readable
        other_results = {
            k: v for k, v in arch_results.items()
            if k != "results"
        }
        # Have a pointer to the separate results file
        other_results["results"] = os.path.abspath(arch_results_path)
        all_other_results[arch] = other_results
    
    results_path = os.path.join(results_dir, "results.json")
    with open(results_path, "w") as f:
        json.dump(all_other_results, f, indent=4)

    print(f"Saved profile results to {results_path}")

### Loading NeRF layer shapes
You can ignore this, it's for populating the profiling results with additional debug information.

In [5]:
def nerf_layer_shapes() -> dict:
    """ Load layer shape info from the pytorch2timeloop converter. Returns mapping of layer ID to shape dict """
    nerf_layer_dir = "workloads/nerf"
    keys_should_be_1 = ["Hdilation", "Hstride", "P", "Q", "R", "S", "Wdilation", "Wstride"]
    layer_shapes = {}
    
    for layer_path in natural_sort(os.listdir(nerf_layer_dir)):
        layer_path = os.path.join(nerf_layer_dir, layer_path)
        layer_id = int(layer_path.split("layer")[1].split(".")[0])

        with open(layer_path, "r") as f:
            layer_config = yaml.safe_load(f)

        instance = layer_config['problem']['instance']
        for key in keys_should_be_1:
            assert instance[key] == 1, f"{key} != 1"
            del instance[key]

        # print(f"{os.path.basename(layer_path)}, layer_id={layer_id}, {instance}")
        assert layer_id not in layer_shapes
        layer_shapes[layer_id] = {"shape": instance}
        
    assert layer_shapes, "layer_shapes should not be empty"
    return layer_shapes

## Profile using Timeloop and Accelergy
I think we can safely ignore the 'unknown module type' warnings.

In [6]:
archs = ["eyeriss_like", "simba_like", "simple_output_stationary", "simple_weight_stationary"]
failed_archs = set()

for arch in archs:
    print(20 * '=')
    print(f"Running {arch}")
    print(20 * '=')
    
    # Profile - you should only need to change batch_size if anything
    try:
        profiler = Profiler(
            top_dir='workloads',
            sub_dir='nerf',
            timeloop_dir=f"designs/{arch}",
            arch_name=arch,
            model=model,
            input_size=(1, 3),
            batch_size=128,  # TODO: adjust this, ICARUS uses 128
            convert_fc=True,
            exception_module_names=[]
        )
        results, summary, layer_summary = profiler.profile()
    except Exception as e:
        # TODO: figure this out https://piazza.com/class/ldf2iof72w51sl/post/44
        traceback.print_exc()
        print(f"ERROR: could not run profiler for {arch}, do not trust these results!")
        failed_archs.add(arch)
        continue
    
    # Add nerf layer shapes to the layer summary
    for layer_id in layer_summary:
        layer_summary[layer_id].update(nerf_layer_shapes()[layer_id])
        
    # Print summary information
    for k, v in summary.items():
        print(f"{k}: {v}")
        
    profile_results[arch] = {
        "results": results,
        "summary": summary,
        "layer_summary": layer_summary,
    }
    save_results()

unknown module type <class 'accelerating_nerfs.models.SinusoidalEncoder'>
unknown module type <class 'accelerating_nerfs.models.SinusoidalEncoder'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.models.MLP'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.models.DenseLayer'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.models.DenseLayer'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.models.MLP'>
unknown module type <class 'accelerating_nerfs.models.NerfMLP'>
unknown module type <class 'accelerating_nerfs.models.VanillaNeRF'>


Running eyeriss_like


running timeloop to get energy and latency...: 0it [00:00, ?it/s]
/usr/local/lib/python3.8/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::unsqueeze". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.8/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::reshape". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.8/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::sin". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
unknown module type <class 'accelerating_nerfs.models.SinusoidalEncoder'>
unknown module type <class 'accelerating_nerfs.models.SinusoidalEncoder'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.models.MLP'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.

total_area: 0.0
total_energy: 1211.45
total_cycle: 1136384.0
num_params: 595844
macs: 593450
activation_size: 2300.0
Saved eyeriss_like results to profile_results/eyeriss_like_results.json
Saved profile results to profile_results/results.json
Running simba_like


running timeloop to get energy and latency...: 100%|██████████| 6/6 [00:42<00:00,  7.12s/it]
Traceback (most recent call last):
  File "/tmp/ipykernel_5534/3720083429.py", line 22, in <module>
    results, summary, layer_summary = profiler.profile()
  File "/home/workspace/notebooks/profiler.py", line 349, in profile
    self.populate_profiled_lib(layer_info)
  File "/home/workspace/notebooks/profiler.py", line 297, in populate_profiled_lib
    info = {key: layer_info[layer_id][key] for key in keys_to_include}
  File "/home/workspace/notebooks/profiler.py", line 297, in <dictcomp>
    info = {key: layer_info[layer_id][key] for key in keys_to_include}
KeyError: 'energy'
unknown module type <class 'accelerating_nerfs.models.SinusoidalEncoder'>
unknown module type <class 'accelerating_nerfs.models.SinusoidalEncoder'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.models.MLP'>
unknown module type <class 'torch.nn.modules.linear

CRITICAL WARNING: /home/workspace/notebooks/designs/simba_like/nerf/layer1/timeloop-mapper.stats.txt does not exist, skipping...
CRITICAL WARNING: /home/workspace/notebooks/designs/simba_like/nerf/layer6/timeloop-mapper.stats.txt does not exist, skipping...
CRITICAL WARNING: /home/workspace/notebooks/designs/simba_like/nerf/layer11/timeloop-mapper.stats.txt does not exist, skipping...
ERROR: could not run profiler for simba_like, do not trust these results!
Running simple_output_stationary


running timeloop to get energy and latency...: 100%|██████████| 6/6 [04:14<00:00, 42.47s/it]
Traceback (most recent call last):
  File "/tmp/ipykernel_5534/3720083429.py", line 22, in <module>
    results, summary, layer_summary = profiler.profile()
  File "/home/workspace/notebooks/profiler.py", line 349, in profile
    self.populate_profiled_lib(layer_info)
  File "/home/workspace/notebooks/profiler.py", line 297, in populate_profiled_lib
    info = {key: layer_info[layer_id][key] for key in keys_to_include}
  File "/home/workspace/notebooks/profiler.py", line 297, in <dictcomp>
    info = {key: layer_info[layer_id][key] for key in keys_to_include}
KeyError: 'energy'
unknown module type <class 'accelerating_nerfs.models.SinusoidalEncoder'>
unknown module type <class 'accelerating_nerfs.models.SinusoidalEncoder'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.models.MLP'>
unknown module type <class 'torch.nn.modules.linear

CRITICAL WARNING: /home/workspace/notebooks/designs/simple_output_stationary/nerf/layer1/timeloop-mapper.stats.txt does not exist, skipping...
CRITICAL WARNING: /home/workspace/notebooks/designs/simple_output_stationary/nerf/layer6/timeloop-mapper.stats.txt does not exist, skipping...
CRITICAL WARNING: /home/workspace/notebooks/designs/simple_output_stationary/nerf/layer11/timeloop-mapper.stats.txt does not exist, skipping...
ERROR: could not run profiler for simple_output_stationary, do not trust these results!
Running simple_weight_stationary


running timeloop to get energy and latency...: 0it [00:00, ?it/s]
/usr/local/lib/python3.8/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::unsqueeze". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.8/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::reshape". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.8/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::sin". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


total_area: 0.0
total_energy: 1143.2300000000002
total_cycle: 4059456.0
num_params: 595844
macs: 593450
activation_size: 2300.0
Saved eyeriss_like results to profile_results/eyeriss_like_results.json
Saved simple_weight_stationary results to profile_results/simple_weight_stationary_results.json
Saved profile results to profile_results/results.json


In [7]:
for arch, arch_results in profile_results.items():
    print(f"===== {arch} =====")
    print(summary)

===== eyeriss_like =====
{'total_area': 0.0, 'total_energy': 1143.2300000000002, 'total_cycle': 4059456.0, 'num_params': 595844, 'macs': 593450, 'activation_size': 2300.0}
===== simple_weight_stationary =====
{'total_area': 0.0, 'total_energy': 1143.2300000000002, 'total_cycle': 4059456.0, 'num_params': 595844, 'macs': 593450, 'activation_size': 2300.0}
